In [3]:
import pandas as pd

# Load the data
train_data = pd.read_csv('/content/fraudTrain.csv')
test_data = pd.read_csv('/content/fraudTest.csv')

# Explore the data
print(train_data.head())
print(train_data.info())
print(train_data.describe())

# Print column names to identify categorical features
print(train_data.columns)


   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street  ...      lat      long  \
0    Banks      F                561 Perry Cove  ...  36.0788  -81.1781   
1     Gill      F  43039 Riley Greens Suite 393  ...  48

In [4]:
train_data = train_data.drop( columns= ['trans_date_trans_time'])
test_data = test_data.drop( columns= ['trans_date_trans_time'])

In [13]:
train_data = train_data.dropna()

In [14]:
train_data.isnull().sum()

,0
Unnamed: 0,0
cc_num,0
merchant,0
category,0
amt,0
first,0
last,0
gender,0
street,0
city,0


In [17]:
test_data = test_data.dropna()
test_data.isnull().sum()

,0
Unnamed: 0,0
cc_num,0
merchant,0
category,0
amt,0
first,0
last,0
gender,0
street,0
city,0


In [15]:
print(train_data)

      Unnamed: 0            cc_num                             merchant  \
0              0  2703186189652095           fraud_Rippin, Kub and Mann   
1              1      630423337322      fraud_Heller, Gutmann and Zieme   
2              2    38859492057661                 fraud_Lind-Buckridge   
3              3  3534093764340240   fraud_Kutch, Hermiston and Farrell   
4              4   375534208663984                  fraud_Keeling-Crist   
...          ...               ...                                  ...   
7809        7809  3558881852140592                   fraud_Bernhard Inc   
7810        7810  6524376634971418                 fraud_Murray-Smitham   
7811        7811  2348245054386329              fraud_Hackett-Lueilwitz   
7812        7812  2283743876903625                  fraud_Koepp-Witting   
7813        7813   213195754007681  fraud_Robel, Cummerata and Prosacco   

           category     amt        first      last gender  \
0          misc_net    4.97     Jennif

In [18]:
print(test_data)

      Unnamed: 0            cc_num                              merchant  \
0              0  2291163933867244                 fraud_Kirlin and Sons   
1              1  3573030041201292                  fraud_Sporer-Keebler   
2              2  3598215285024754  fraud_Swaniawski, Nitzsche and Welch   
3              3  3591919803438423                     fraud_Haley Group   
4              4  3526826139003047                 fraud_Johnston-Casper   
...          ...               ...                                   ...   
7792        7792      577588686219                   fraud_Langworth LLC   
7793        7793   377113842678100                fraud_Fritsch and Sons   
7794        7794  6538891242532018          fraud_Torp, Muller and Borer   
7795        7795     4658117080136                     fraud_Haley Group   
7796        7796  2720830304681674                   fraud_Kozey-Kuhlman   

            category     amt    first        last gender  \
0      personal_care    2.8

In [19]:
# Replace these with actual categorical columns identified from the dataset
categorical_features = ['merchant', 'category' , 'first' , 'last' , 'gender' , 'street' , 'city' , 'state' , 'job' , 'dob' , 'trans_num']  # Replace with actual categorical columns
numerical_features = [col for col in train_data.columns if col not in categorical_features + ['is_fraud']]
# Replace these with actual categorical columns identified from the dataset
categorical_features = ['merchant', 'category' , 'first' , 'last' , 'gender' , 'street' , 'city' , 'state' , 'job' , 'dob' , 'trans_num']  # Replace with actual categorical columns
numerical_features = [col for col in train_data.columns if col not in categorical_features + ['is_fraud']]


In [20]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [21]:
# Preprocess and prepare the data
X_train = train_data.drop(columns=['is_fraud'])
y_train = train_data['is_fraud']
X_test = test_data.drop(columns=['is_fraud'])
y_test = test_data['is_fraud']

# Fit and transform the training data, transform the test data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


In [22]:
from sklearn.model_selection import train_test_split

# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_preprocessed, y_train, test_size=0.2, random_state=42)


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Initialize models
log_reg = LogisticRegression()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()

# Train models
log_reg.fit(X_train_split, y_train_split)
decision_tree.fit(X_train_split, y_train_split)
random_forest.fit(X_train_split, y_train_split)


RandomForestClassifier()

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to evaluate models
def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    return accuracy, precision, recall, f1

# Evaluate models
log_reg_metrics = evaluate_model(log_reg, X_val_split, y_val_split)
decision_tree_metrics = evaluate_model(decision_tree, X_val_split, y_val_split)
random_forest_metrics = evaluate_model(random_forest, X_val_split, y_val_split)

print("Logistic Regression metrics:", log_reg_metrics)
print("Decision Tree metrics:", decision_tree_metrics)
print("Random Forest metrics:", random_forest_metrics)


Logistic Regression metrics: (0.9974408189379399, 0.9090909090909091, 0.7692307692307693, 0.8333333333333333)
Decision Tree metrics: (0.9987204094689699, 0.9230769230769231, 0.9230769230769231, 0.9230769230769231)
Random Forest metrics: (0.999360204734485, 1.0, 0.9230769230769231, 0.9600000000000001)


In [25]:
# Assuming Random Forest performs the best
best_model = random_forest


In [26]:
# Predict on test data
y_test_pred = best_model.predict(X_test_preprocessed)

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(best_model, X_test_preprocessed, y_test)
print("Test set metrics:", test_accuracy, test_precision, test_recall, test_f1)


Test set metrics: 0.9966653841220983 0.0 0.0 0.0
